# 1. Data API

In [6]:
import tensorflow as tf
def cls():
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

In [9]:
#########################################
# Dataset API always return new dataset #
#########################################
x = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [10]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [16]:
X_nested = {"a": ([1, 2, 3], [4, 5, 6]), "b": [7, 8, 9]}
dataset = tf.data.Dataset.from_tensor_slices(X_nested)
for idx, item in enumerate(dataset):
    print(idx+1,'\n\t', item)

1 
	 {'a': (<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=4>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>}
2 
	 {'a': (<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>}
3 
	 {'a': (<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=6>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>}


### 1.1. Chaining Tranformations

In [25]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset.repeat(3).batch(7) # repeat each value 3 times and sort randomly then create batch
for idx, item in enumerate(dataset):
    print(idx+1,'\t', item)

1 	 tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
2 	 tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
3 	 tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
4 	 tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
5 	 tf.Tensor([8 9], shape=(2,), dtype=int32)


In [26]:
dataset = dataset.map(lambda x: x*2) # x is a single batch
for idx, item in enumerate(dataset):
    print(idx+1,'\t', item)

1 	 tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
2 	 tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
3 	 tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
4 	 tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
5 	 tf.Tensor([16 18], shape=(2,), dtype=int32)


In [28]:
dataset = dataset.filter(lambda x: tf.reduce_sum(x) > 50)
for idx, item in enumerate(dataset):
    print(idx+1,'\t', item)

1 	 tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
2 	 tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
3 	 tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
